## Prompt Optimization

Automatically create a prompt that is optimized to the model and evaluation metric.

In [2]:
#%pip install --quiet dspy

In [3]:
#GEMINI="gemini-2.0-flash"
#OPENAI="gpt-4o-mini"
CLAUDE="claude-3-7-sonnet-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
#assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
#       "Please specify the GEMINI_API_KEY access token in keys.env file"
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"
assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
       "Please specify the OPENAI_API_KEY access token in keys.env file"

## Data

For simplicity, we'll hardcode the text of the marketing blurbs for a couple of O'Reilly books.

In [4]:
aieng_text="""
Recent breakthroughs in AI have not only increased demand for AI products, they've also lowered the barriers to entry for those who want to build AI products. The model-as-a-service approach has transformed AI from an esoteric discipline into a powerful development tool that anyone can use. Everyone, including those with minimal or no prior AI experience, can now leverage AI models to build applications. In this book, author Chip Huyen discusses AI engineering: the process of building applications with readily available foundation models.

The book starts with an overview of AI engineering, explaining how it differs from traditional ML engineering and discussing the new AI stack. The more AI is used, the more opportunities there are for catastrophic failures, and therefore, the more important evaluation becomes. This book discusses different approaches to evaluating open-ended models, including the rapidly growing AI-as-a-judge approach.

AI application developers will discover how to navigate the AI landscape, including models, datasets, evaluation benchmarks, and the seemingly infinite number of use cases and application patterns. You'll learn a framework for developing an AI application, starting with simple techniques and progressing toward more sophisticated methods, and discover how to efficiently deploy these applications.

Understand what AI engineering is and how it differs from traditional machine learning engineering
Learn the process for developing an AI application, the challenges at each step, and approaches to address them
Explore various model adaptation techniques, including prompt engineering, RAG, fine-tuning, agents, and dataset engineering, and understand how and why they work
Examine the bottlenecks for latency and cost when serving foundation models and learn how to overcome them
Choose the right model, dataset, evaluation benchmarks, and metrics for your needs
Chip Huyen works to accelerate data analytics on GPUs at Voltron Data. Previously, she was with Snorkel AI and NVIDIA, founded an AI infrastructure startup, and taught Machine Learning Systems Design at Stanford. She's the author of the book Designing Machine Learning Systems, an Amazon bestseller in AI.

AI Engineering builds upon and is complementary to Designing Machine Learning Systems (O'Reilly).
"""

In [5]:
mldp_text="""
The design patterns in this book capture best practices and solutions to recurring problems in machine learning. The authors, three Google engineers, catalog proven methods to help data scientists tackle common problems throughout the ML process. These design patterns codify the experience of hundreds of experts into straightforward, approachable advice.

In this book, you will find detailed explanations of 30 patterns for data and problem representation, operationalization, repeatability, reproducibility, flexibility, explainability, and fairness. Each pattern includes a description of the problem, a variety of potential solutions, and recommendations for choosing the best technique for your situation.

You'll learn how to:

Identify and mitigate common challenges when training, evaluating, and deploying ML models
Represent data for different ML model types, including embeddings, feature crosses, and more
Choose the right model type for specific problems
Build a robust training loop that uses checkpoints, distribution strategy, and hyperparameter tuning
Deploy scalable ML systems that you can retrain and update to reflect new data
Interpret model predictions for stakeholders and ensure models are treating users fairly
"""

In [6]:
for in_text in [aieng_text, mldp_text]:
    print(f"{len(in_text.splitlines())} lines {len(in_text)} characters")

15 lines 2323 characters
13 lines 1240 characters


Let's set the targets based on the book that was published later.

In [7]:
TARGET_MAX_LINES = 15
TARGET_MAX_CHARS = 2400
TARGET_NUM_OBJECTIVES = 6

## Extract the pieces of the marketing description

In [8]:
import dspy
lm = dspy.LM(CLAUDE, api_key=os.environ['ANTHROPIC_API_KEY'])
dspy.configure(lm=lm)

In [14]:
from typing import List
class BlurbExtraction(dspy.Signature):
    text: str = dspy.InputField(desc="Text from backcover")
    about_topic: str = dspy.OutputField(desc="Extracted line about why the topic of book is worth learning")
    about_book: str = dspy.OutputField(desc="Extracted line about what book contains")
    target_audience: List[str] = dspy.OutputField(desc="Inferred roles such as Data Engineer, Data Analyst that the book is for")
    learning_objectives: List[str] = dspy.OutputField(desc="Extracted learning objectives")

In [15]:
module = dspy.ChainOfThought(BlurbExtraction)
cover_info = module(text=aieng_text)
cover_info

Prediction(
    reasoning='Let me analyze this book description to extract the key information:\n\n1. About the topic: The text explains that AI engineering is now accessible to everyone, even those without prior AI experience, due to recent breakthroughs and the model-as-a-service approach.\n\n2. About the book: The book covers AI engineering as a process of building applications with foundation models, starting with an overview and then diving into evaluation, navigating the AI landscape, and a framework for developing AI applications.\n\n3. Target audience: While not explicitly stated, the book appears to target AI application developers, those with minimal AI experience who want to build AI products, and potentially traditional ML engineers looking to transition to AI engineering.\n\n4. Learning objectives: These can be extracted from the bullet points and the description of what readers will learn, including understanding AI engineering, learning the development process, exploring

In [18]:
cover_info.toDict()

{'reasoning': 'Let me analyze this book description to extract the key information:\n\n1. About the topic: The text explains that AI engineering is now accessible to everyone, even those without prior AI experience, due to recent breakthroughs and the model-as-a-service approach.\n\n2. About the book: The book covers AI engineering as a process of building applications with foundation models, starting with an overview and then diving into evaluation, navigating the AI landscape, and a framework for developing AI applications.\n\n3. Target audience: While not explicitly stated, the book appears to target AI application developers, those with minimal AI experience who want to build AI products, and potentially traditional ML engineers looking to transition to AI engineering.\n\n4. Learning objectives: These can be extracted from the bullet points and the description of what readers will learn, including understanding AI engineering, learning the development process, exploring model adapt

## Improve the marketing description components

In [21]:
## Improve the pieces
from typing import Dict, Any
class BlurbImprovement(dspy.Signature):
    current_cover: Dict[str, Any] = dspy.InputField(descr="Current information on book")
    about_topic: str = dspy.OutputField(desc="More catchy statement why topic is worth learning")
    about_book: str = dspy.OutputField(desc="More appealing (to target audience) description of book contents")
    target_audience: List[str] = dspy.OutputField(desc="more aspirational list of roles. Instead of programmer, say software engineer. Restrict to top 3.")
    learning_objectives: List[str] = dspy.OutputField(desc="Learning objectives rephrased or reordered to be more appealing to target audience. Exactly 6.")    
    
module = dspy.ChainOfThought(BlurbImprovement)
improved_cover = module(current_cover=cover_info.toDict())
improved_cover

Prediction(
    reasoning='I need to make the book description more compelling and aspirational while maintaining accuracy. For the "about topic" section, I\'ll create a more catchy statement highlighting the transformative nature of AI engineering. For the "about book" section, I\'ll craft a more engaging description that appeals to ambitious professionals. The target audience should be more aspirational, so I\'ll elevate the roles and limit to the top 3. For learning objectives, I\'ll rephrase them to emphasize career advancement and practical skills, keeping exactly 6 items.',
    about_topic="AI engineering is experiencing a revolutionary moment. What was once the domain of specialized researchers is now a powerful toolkit accessible to everyone. With foundation models and AI services, you can build sophisticated AI applications that were unimaginable just a few years ago—even if you've never worked with AI before.",
    about_book="This practical guide takes you on a journey from 

In [37]:
from dataclasses import dataclass

@dataclass
class Blurb:
    about_topic: str
    about_book: str
    target_audience: List[str]
    learning_objectives: List[str]
    
    def __init__(self, d: dict):
        self.about_topic = d['about_topic']
        self.about_book = d['about_book']
        self.target_audience = d['target_audience']
        self.learning_objectives = d['learning_objectives']
    
    def toMarketingCopy(self) -> str:
        nl = '\n'
        return f"""{self.about_topic}
        
{self.about_book}
        
{', '.join(self.target_audience)} will learn how to:
{nl.join(['* ' + objective for objective in self.learning_objectives])}
        """

print(Blurb(improved_cover.toDict()).toMarketingCopy())

AI engineering is experiencing a revolutionary moment. What was once the domain of specialized researchers is now a powerful toolkit accessible to everyone. With foundation models and AI services, you can build sophisticated AI applications that were unimaginable just a few years ago—even if you've never worked with AI before.
        
This practical guide takes you on a journey from AI novice to confident AI engineer. You'll master the new AI stack and learn a systematic framework for building production-ready AI applications. Starting with core concepts, you'll quickly progress to advanced techniques for customizing models, optimizing performance, and deploying solutions that deliver real business value. Through hands-on examples and expert guidance, you'll gain the skills to transform innovative ideas into working AI products that solve real-world problems.
        
Software Engineers, AI Product Developers, Technology Leaders will learn:
* Master the fundamentals of AI engineering 

## Build an evaluator for the final output.

This is the metric that will be optimized

In [57]:
class BlurbScore(dspy.Signature):
    blurb: Blurb = dspy.InputField()
    topic_score: float = dspy.OutputField(desc="0-1 on how appealing the statement on why topic is worth learning")
    contents_score: float = dspy.OutputField(desc="0-1 on how appealing (to target audience) the book contents are")
    objectives_score: List[float] = dspy.OutputField(desc="0-1 score of how appealing the learning objective is to target audience")
   
module = dspy.ChainOfThought(BlurbScore)
score_pred = module(blurb = Blurb(improved_cover.toDict()))
score_pred

Prediction(
    reasoning='Let me evaluate this book blurb from the perspective of the target audience (Software Engineers, AI Product Developers, and Technology Leaders):\n\nTopic Appeal:\nThe blurb positions AI engineering as undergoing a revolutionary transformation, making previously specialized technology accessible to everyone. This is highly appealing because it suggests an opportunity for the target audience to gain valuable skills in a rapidly growing field. The framing that you can build sophisticated AI applications "even if you\'ve never worked with AI before" makes the topic approachable and timely. The topic score should be very high as it addresses a cutting-edge technology with practical applications.\n\nContents Appeal:\nThe book promises a journey from "AI novice to confident AI engineer" with a systematic framework for building production-ready applications. This is extremely valuable to the target audience who likely need practical, implementation-focused guidance. 

In [54]:
def calc_aggregate_score(blurb: Blurb, p: dspy.Prediction) -> float:   
    result = (p.topic_score * 10 + p.contents_score * 10 + sum(p.objectives_score)) / (20 + len(p.objectives_score))
    ## cut 0.1 for every line beyond MAX_LINES, etc.
    marketing_copy = blurb.toMarketingCopy()
    num_lines = len(marketing_copy.splitlines())
    num_chars = len(marketing_copy)
    num_objectives = len(blurb.learning_objectives)
    if num_lines > TARGET_MAX_LINES:
        result -= 0.1 * (num_lines - TARGET_MAX_LINES)
    if num_chars > TARGET_MAX_CHARS:
        result -= 0.01 * (num_chars - TARGET_MAX_CHARS)
    if num_objectives > TARGET_NUM_OBJECTIVES:
        result -= 0.1 * (num_objectives - TARGET_NUM_OBJECTIVES)
    if result < 0:
        result = 0
    return result

In [58]:
calc_aggregate_score(blurb, score_pred)

0.9142307692307692

## Combine into a module that can be optimized all at once

In [65]:
class BlurbPipeline(dspy.Module):
    def __init__(self):
        self.extract_info = dspy.ChainOfThought(BlurbExtraction)
        self.improve_blurb = dspy.ChainOfThought(BlurbImprovement)
        self.scorer = dspy.ChainOfThought(BlurbScore)
        
    def forward(self, in_text: str) -> (Blurb, float):
        cover_info = self.extract_info(text=in_text)
        improved_cover = self.improve_blurb(current_cover=cover_info.toDict())
        blurb = Blurb(improved_cover.toDict())
        score_pred = self.scorer(blurb=blurb)
        return blurb, calc_aggregate_score(blurb, score_pred)
    
program = BlurbPipeline()
blurb, score = program(in_text=aieng_text)
print(score, '\n', blurb.toMarketingCopy())

0.9142307692307692 
 AI engineering is experiencing a revolutionary moment. What was once the domain of specialized researchers is now a powerful toolkit accessible to everyone. With foundation models and AI services, you can build sophisticated AI applications that were unimaginable just a few years ago—even if you've never worked with AI before.
        
This practical guide takes you on a journey from AI novice to confident AI engineer. You'll master the new AI stack and learn a systematic framework for building production-ready AI applications. Starting with core concepts, you'll quickly progress to advanced techniques for customizing models, optimizing performance, and deploying solutions that deliver real business value. Through hands-on examples and expert guidance, you'll gain the skills to transform innovative ideas into working AI products that solve real-world problems.
        
Software Engineers, AI Product Developers, Technology Leaders will learn:
* Master the fundamenta

## Do it several times and choose the best

(dspy supports other optimization options)

In [74]:
import dspy

def score_reward(args, pred) -> float:
    blurb, score = pred
    print(f"""**** {score}
{blurb.toMarketingCopy()}
******
    """)
    return score

optimized_pipeline = dspy.BestOfN(
    module=BlurbPipeline(),
    N=10,
    reward_fn=score_reward,
    threshold=0.95
)

blurb, score = optimized_pipeline(
    in_text=aieng_text
)
print(score, '\n', blurb.toMarketingCopy())

**** 0.9142307692307692
AI engineering is experiencing a revolutionary moment. What was once the domain of specialized researchers is now a powerful toolkit accessible to everyone. With foundation models and AI services, you can build sophisticated AI applications that were unimaginable just a few years ago—even if you've never worked with AI before.
        
This practical guide takes you on a journey from AI novice to confident AI engineer. You'll master the new AI stack and learn a systematic framework for building production-ready AI applications. Starting with core concepts, you'll quickly progress to advanced techniques for customizing models, optimizing performance, and deploying solutions that deliver real business value. Through hands-on examples and expert guidance, you'll gain the skills to transform innovative ideas into working AI products that solve real-world problems.
        
Software Engineers, AI Product Developers, Technology Leaders will learn:
* Master the fundame